In [0]:
!java -version

openjdk version "1.8.0_275"
OpenJDK Runtime Environment (Zulu 8.50.0.51-CA-linux64) (build 1.8.0_275-b01)
OpenJDK 64-Bit Server VM (Zulu 8.50.0.51-CA-linux64) (build 25.275-b01, mixed mode)

In [0]:
sc.version

Out[53]: '3.1.0'

In [0]:
#import math

from pyspark.sql import Row

# https://spark.apache.org/docs/latest/ml-classification-regression.html
#from pyspark.ml import Pipeline
#from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import PCA
#from pyspark.ml.feature import IDF, HashingTF, Tokenizer
from pyspark.ml.linalg import Vectors
#from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import FMClassifier
#from pyspark.ml.clustering import KMeans
#from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#import pandas as pd
#import matplotlib.pylab as plt
#%matplotlib inline

In [0]:
df = table("cadastro_nomes_sexo")
#df = spark.sql("select * from cadastro_nomes_sexo")

In [0]:
df.printSchema()

root
-- PNOME: string (nullable = true)
-- SEXO: string (nullable = true)

In [0]:
df.show(10)

+---------+----+
 PNOME|SEXO|
+---------+----+
 MARIA| F|
 JOSE| M|
 ANA| F|
 JOAO| M|
 CARLOS| M|
 PAULO| M|
 ANTONIO| M|
 LUIZ| M|
FRANCISCO| M|
 MARCELO| M|
+---------+----+
only showing top 10 rows

In [0]:
rdd = df.rdd
#rdd = df.rdd.map(list)
#rdd = df.rdd.map(tuple)
rdd

Out[58]: MapPartitionsRDD[1212] at javaToPython at NativeMethodAccessorImpl.java:0

In [0]:
type(rdd)

Out[59]: pyspark.rdd.RDD

In [0]:
rdd.take(5)

Out[60]: [Row(PNOME='MARIA', SEXO='F'),
 Row(PNOME='JOSE', SEXO='M'),
 Row(PNOME='ANA', SEXO='F'),
 Row(PNOME='JOAO', SEXO='M'),
 Row(PNOME='CARLOS', SEXO='M')]

In [0]:
rdd2 = rdd.map(lambda x: Row(
  nome=x['PNOME'],
  sexo={'F': 1, 'M': 2, 'X': 0}[x['SEXO']]
))
rdd2

Out[61]: PythonRDD[1214] at RDD at PythonRDD.scala:58

In [0]:
rdd2.take(5)

Out[62]: [Row(nome='MARIA', sexo=1),
 Row(nome='JOSE', sexo=2),
 Row(nome='ANA', sexo=1),
 Row(nome='JOAO', sexo=2),
 Row(nome='CARLOS', sexo=2)]

In [0]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

def incluir_letras(row):
  row_dict = row.asDict()
  rev = row_dict['nome'][::-1]
  tam = len(rev)
  letras = [0] * MAIOR_QTDE_LETRAS
  
  for i in range(tam):
    letras[i] = ord(rev[i]) - 64
  row_dict['letras'] = letras
  
  #for i in range(MAIOR_QTDE_LETRAS):
  #  col = 'L' + str(i)
  #  val = ord(rev[i]) - 64 if tam > i else 0
  #  row_dict[col] = val
    
  newrow = Row(**row_dict)
  return newrow

In [0]:
rdd3 = rdd2.map(incluir_letras)
rdd3

Out[64]: PythonRDD[1216] at RDD at PythonRDD.scala:58

In [0]:
rdd3.take(5)

Out[65]: [Row(nome='MARIA', sexo=1, letras=[1, 9, 18, 1, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOSE', sexo=2, letras=[5, 19, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='ANA', sexo=1, letras=[1, 14, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='JOAO', sexo=2, letras=[15, 1, 15, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 Row(nome='CARLOS', sexo=2, letras=[19, 15, 12, 18, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [0]:
df3 = rdd3.toDF()
df3

Out[66]: DataFrame[nome: string, sexo: bigint, letras: array<bigint>]

In [0]:
df3.select("nome", "sexo").describe().show()

+-------+------+------------------+
summary| nome| sexo|
+-------+------+------------------+
 count| 48970| 48970|
 mean| NaN|1.2335919950990402|
 stddev| null|0.7059859521809757|
 min| AARAO| 0|
 max|ZYSMAN| 2|
+-------+------+------------------+

In [0]:
# Criando um LabeledPoint (target, Vector[features])
def converter_para_ponto_rotulado(row):
    obj = (row["sexo"], Vectors.dense(row["letras"]), row["nome"])
    return obj

In [0]:
rdd4 = rdd3.map(converter_para_ponto_rotulado)
rdd4

Out[69]: PythonRDD[1229] at RDD at PythonRDD.scala:58

In [0]:
rdd4.take(5)

Out[70]: [(1,
 DenseVector([1.0, 9.0, 18.0, 1.0, 13.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 'MARIA'),
 (2,
 DenseVector([5.0, 19.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 'JOSE'),
 (1,
 DenseVector([1.0, 14.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 'ANA'),
 (2,
 DenseVector([15.0, 1.0, 15.0, 10.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 'JOAO'),
 (2,
 DenseVector([19.0, 15.0, 12.0, 18.0, 1.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 'CARLOS')]

In [0]:
df4 = rdd4.toDF(["real", "features", "nome"])
df4

Out[71]: DataFrame[real: bigint, features: vector, nome: string]

In [0]:
df4.show(5, truncate=False)

+----+---------------------------------------------------------------------+------+
real|features |nome |
+----+---------------------------------------------------------------------+------+
1 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |MARIA |
2 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |JOSE |
1 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |ANA |
2 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |JOAO |
2 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|CARLOS|
+----+---------------------------------------------------------------------+------+
only showing top 5 rows

In [0]:
pca = PCA(
  k = 10,
  inputCol = "features",
  outputCol = "pcaFeatures"
)
pcaModel = pca.fit(df4)
pcaResult = pcaModel.transform(df4).select("nome", "real", "pcaFeatures")
pcaResult.show(5)

+------+----+--------------------+
 nome|real| pcaFeatures|
+------+----+--------------------+
 MARIA| 1|[1.50989098280335...|
 JOSE| 2|[-1.9188485990558...|
 ANA| 1|[0.60529476692447...|
 JOAO| 2|[-5.2919527601426...|
CARLOS| 2|[-4.2915224156896...|
+------+----+--------------------+
only showing top 5 rows

In [0]:
# usar redução de dimensionalidade PCA
#df5 = pcaResult.withColumnRenamed("pcaFeatures", "features")

# não usar PCA
df5 = df4

df5.show(5, truncate=False)

+----+---------------------------------------------------------------------+------+
real|features |nome |
+----+---------------------------------------------------------------------+------+
1 |[1.0,9.0,18.0,1.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |MARIA |
2 |[5.0,19.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |JOSE |
1 |[1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |ANA |
2 |[15.0,1.0,15.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0] |JOAO |
2 |[19.0,15.0,12.0,18.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]|CARLOS|
+----+---------------------------------------------------------------------+------+
only showing top 5 rows

In [0]:
(dados_treino, dados_teste) = df5.randomSplit([0.7, 0.3], seed=42)
print("dados de treino:", dados_treino.count())
print("dados de teste: ", dados_teste.count())

dados de treino: 34316
dados de teste: 14654

In [0]:
classificador = MultilayerPerceptronClassifier(
  maxIter=100,
  layers=[4, 5, 4, 3],
  blockSize=128,
  seed=1234,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)
modelo = classificador.fit(dados_treino)
modelo

Out[75]: MultilayerPerceptronClassificationModel: uid=MultilayerPerceptronClassifier_0efc48030887, numLayers=4, numClasses=3, numFeatures=4

In [0]:
#classificador = GBTClassifier( # só classificação binária!
#classificador = DecisionTreeClassifier(
classificador = RandomForestClassifier(
  maxDepth=10,
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)
modelo = classificador.fit(dados_treino)
modelo

Out[93]: RandomForestClassificationModel: uid=RandomForestClassifier_1d7104e7ef4f, numTrees=20, numClasses=3, numFeatures=16

In [0]:
#classificador = FMClassifier(
  #stepSize=0.001,
#classificador = NaiveBayes(
  #smoothing=1.0,
  #modelType="multinomial",
#classificador = LinearSVC(
  #maxIter=10,
  #regParam=0.1,
classificador = LogisticRegression(
  #regParam=1.0,
  #maxIter=1000,
# ...
  labelCol="real",
  featuresCol="features",
  predictionCol="previsto"
)
#modelo = classificador.fit(dados_treino)
#modelo
#print("Coefficients: " + str(lsvcModel.coefficients))
#print("Intercept: " + str(lsvcModel.intercept))

In [0]:
previsoes = modelo.transform(dados_teste)
previsoes.select("nome", "real", "previsto").show(10)

+--------+----+--------+
 nome|real|previsto|
+--------+----+--------+
ABINADAB| 0| 0.0|
 AULIAB| 0| 0.0|
 MAB| 0| 0.0|
 ZAINAB| 0| 1.0|
 CALEB| 0| 0.0|
 TALEB| 0| 0.0|
 KALLEB| 0| 0.0|
 ABIB| 0| 0.0|
 NACIB| 0| 0.0|
 ADIB| 0| 0.0|
+--------+----+--------+
only showing top 10 rows

In [0]:
# acurácia do modelo
avaliador = MulticlassClassificationEvaluator(
    predictionCol = "previsto", 
    labelCol = "real",
    metricName = "accuracy"
)
acuracia = avaliador.evaluate(previsoes) * 100
print("acurácia: %.2f %%" % acuracia)

acurácia: 93.02 %

In [0]:
# matriz de confusão
previsoes.groupBy("real", "previsto").count().show()

+----+--------+-----+
real|previsto|count|
+----+--------+-----+
 1| 0.0| 63|
 0| 1.0| 323|
 1| 2.0| 158|
 2| 0.0| 48|
 2| 2.0| 5709|
 0| 0.0| 1523|
 1| 1.0| 6399|
 0| 2.0| 385|
 2| 1.0| 46|
+----+--------+-----+

In [0]:
previsoes.filter("real = 2").filter("real != previsto").select("nome", "real", "previsto").show(10)

+---------+----+--------+
 nome|real|previsto|
+---------+----+--------+
LOURILLAC| 2| 0.0|
 ISAC| 2| 0.0|
 MELZAC| 2| 0.0|
 TAWFIC| 2| 0.0|
 ELIC| 2| 0.0|
 DOMINIC| 2| 0.0|
 MONNIC| 2| 1.0|
 MONIC| 2| 0.0|
 RIC| 2| 0.0|
 EDRIC| 2| 1.0|
+---------+----+--------+
only showing top 10 rows

In [0]:
previsoes.filter("real = 1").filter("real != previsto").select("nome", "real", "previsto").show(10)

+-------+----+--------+
 nome|real|previsto|
+-------+----+--------+
 INGRID| 1| 0.0|
 RABAH| 1| 0.0|
ASSAIAH| 1| 0.0|
 MOSIAH| 1| 0.0|
 NOAH| 1| 0.0|
 MAYAH| 1| 0.0|
 IZAH| 1| 0.0|
 EDITH| 1| 0.0|
 JUDITH| 1| 0.0|
 RUTH| 1| 0.0|
+-------+----+--------+
only showing top 10 rows

In [0]:
# Encontrando a correlação entre a variável
# target com as variáveis preditoras
for i in carrosDF.columns:
    if not(isinstance(carrosDF.select(i).take(1)[0][0], str)) :
        print("Correlação da variável MPG com", i, 
              carrosDF.stat.corr('MPG', i))

In [0]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [0]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [0]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [0]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

In [0]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 87.45
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

In [0]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [0]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

In [0]:
model = models['CB'][0]
model

In [0]:
model.fit(X, y)

In [0]:
y_pred = model.predict(X)
y_pred

In [0]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

In [0]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

In [0]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

In [0]:
dados[dados['REAL'] != dados['PREV']].head(20)